# 📊 Análisis Exploratorio de Datos para Mantenimiento Predictivo del Motocompresor C-5080

**Universidad:** Escuela Militar de Ingeniería - Unidad Académica Santa Cruz  
**Carrera:** Ingeniería Mecatrónica
**Estudiante:** Miguel Salazar  
**Proyecto:** Sistema de Mantenimiento Predictivo Basado en IA para Motocompresores de Gas Natural  
**Equipo Objetivo:** Motocompresor C-5080 (Motor Waukesha L 7042 GSI + Compresor Cooper-Bessemer AMA-4)  
**Empresa:** YPFB Andina S.A.  

---

## 🎯 Introducción

Este documento presenta un **Análisis Exploratorio de Datos (EDA)** exhaustivo como parte del desarrollo de un sistema de mantenimiento predictivo basado en inteligencia artificial. El objetivo principal es comprender a fondo las características de los datos operacionales del motocompresor C-5080, identificar patrones, anomalías, correlaciones entre variables y establecer relaciones entre las lecturas de sensores y los eventos de falla históricos.

### 🏭 Contexto Industrial

El motocompresor C-5080 es un equipo crítico en la planta de YPFB Andina que actualmente opera bajo una estrategia de mantenimiento preventivo basada en horas de operación. Esta estrategia ha demostrado ser ineficaz, resultando en:

- ⚠️ Numerosas paradas no programadas
- 💰 Pérdidas económicas significativas
- 🔧 Mantenimientos innecesarios o tardíos
- 📉 Baja eficiencia operacional

### 🎯 Objetivos del EDA

1. **Caracterizar** los datos de sensores y su calidad
2. **Identificar** patrones operacionales normales y anómalos
3. **Establecer** correlaciones entre variables de proceso
4. **Detectar** precursores de fallas en los datos históricos
5. **Evaluar** la viabilidad de implementar modelos predictivos
6. **Proporcionar** insights para la ingeniería de características

### 📋 Metodología

El análisis seguirá una metodología estructurada en 8 etapas:
1. Carga e inspección inicial de datos
2. Limpieza y validación de datos
3. Análisis estadístico descriptivo
4. Análisis univariado (distribuciones individuales)
5. Análisis bivariado y multivariado (correlaciones)
6. Análisis de series temporales
7. Integración con historial de fallas
8. Síntesis y conclusiones

---

In [44]:
# 📚 Importación de Librerías Esenciales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from datetime import datetime, timedelta
import glob
from scipy import stats
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
warnings.filterwarnings('ignore')

# Configuración de pandas para mostrar más columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

%matplotlib inline
print("✅ Librerías importadas exitosamente")
print(f"📊 Pandas versión: {pd.__version__}")
print(f"🔢 NumPy versión: {np.__version__}")
print(f"📈 Matplotlib versión: {plt.__version__ if hasattr(plt, '__version__') else 'N/A'}")
print(f"🎨 Seaborn versión: {sns.__version__}")

✅ Librerías importadas exitosamente
📊 Pandas versión: 2.3.1
🔢 NumPy versión: 2.3.2
📈 Matplotlib versión: N/A
🎨 Seaborn versión: 0.13.2


---

## 1. 📁 Carga e Inspección Inicial de Datos

En esta sección cargaremos todos los archivos de datos disponibles, incluyendo los datos de sensores del motocompresor y el historial de fallas. Es fundamental comprender la estructura, formato y contenido de cada dataset antes de proceder con el análisis.

### 🔍 Identificación de Archivos de Datos

In [45]:
# 📂 Definir rutas de datos
data_path = Path('data/raw')
eventos_path = Path('eventos')

print("🔍 INVENTARIO DE ARCHIVOS DE DATOS")
print("=" * 50)

# Buscar archivos de datos de sensores
sensor_files = list(data_path.glob('*.xls')) + list(data_path.glob('*.xlsx'))
sensor_files = [f for f in sensor_files if 'Historial' not in f.name and f.suffix in ['.xls', '.xlsx']]

# Buscar archivo de historial de fallas
historial_files = list(data_path.glob('*Historial*.xlsx')) + list(eventos_path.glob('*Historial*.xlsx'))

print(f"📊 Archivos de Sensores Encontrados: {len(sensor_files)}")
for i, file in enumerate(sensor_files[:10], 1):  # Mostrar solo los primeros 10
    file_size = file.stat().st_size / (1024*1024)  # Tamaño en MB
    print(f"   {i:2d}. {file.name:<30} ({file_size:.2f} MB)")
    
if len(sensor_files) > 10:
    print(f"   ... y {len(sensor_files) - 10} archivos más")

print(f"\n📋 Archivos de Historial de Fallas: {len(historial_files)}")
for file in historial_files:
    file_size = file.stat().st_size / (1024*1024)
    print(f"   • {file.name} ({file_size:.2f} MB)")

# Ordenar archivos de sensores cronológicamente
sensor_files_sorted = sorted(sensor_files, key=lambda x: x.name)

print(f"\n📈 Rango Temporal de Datos:")
if sensor_files_sorted:
    print(f"   Primer archivo: {sensor_files_sorted[0].name}")
    print(f"   Último archivo: {sensor_files_sorted[-1].name}")

print(f"\n💾 Tamaño Total de Datos: {sum(f.stat().st_size for f in sensor_files + historial_files) / (1024*1024):.2f} MB")

🔍 INVENTARIO DE ARCHIVOS DE DATOS
📊 Archivos de Sensores Encontrados: 28
    1. 01-2024.xls                    (0.48 MB)
    2. 01-2025.xls                    (0.47 MB)
    3. 02-2024..xls                   (0.45 MB)
    4. 02-2025.xls                    (0.43 MB)
    5. 03-2024..xls                   (0.48 MB)
    6. 03-2025.xls                    (0.47 MB)
    7. 04-2024..xls                   (0.46 MB)
    8. 04-2025.xls                    (0.45 MB)
    9. 05-2024..xls                   (0.47 MB)
   10. 06-2024..xls                   (0.46 MB)
   ... y 18 archivos más

📋 Archivos de Historial de Fallas: 1
   • Historial C1 RGD.xlsx (0.04 MB)

📈 Rango Temporal de Datos:
   Primer archivo: 01-2024.xls
   Último archivo: 9-2023...xls

💾 Tamaño Total de Datos: 13.08 MB


### 📊 Carga de Datos de Sensores

Procederemos a cargar una muestra representativa de los archivos de sensores para entender su estructura y contenido. Comenzaremos con algunos archivos para optimizar el rendimiento del análisis.

In [46]:
# 📥 Función mejorada para cargar archivos de sensores con estructura compleja
def load_sensor_file(file_path, max_rows=None):
    """
    Carga un archivo de sensores con manejo inteligente de estructura compleja de Excel
    """
    try:
        # Primero, cargar el archivo completo para analizar su estructura
        if file_path.suffix == '.xlsx':
            df_raw = pd.read_excel(file_path, header=None, engine='openpyxl')
        else:
            df_raw = pd.read_excel(file_path, header=None, engine='xlrd')
        
        # Buscar la fila donde comienzan los datos reales
        # Buscar filas que contengan 'COMPRESOR' o 'MOTOR' como indicadores
        header_row = None
        data_start_row = None
        
        for idx, row in df_raw.iterrows():
            row_str = ' '.join(str(cell) for cell in row if pd.notna(cell))
            if 'COMPRESOR' in row_str.upper() or 'MOTOR' in row_str.upper():
                header_row = idx
                data_start_row = idx + 1
                break
        
        # Si no encontramos indicadores específicos, buscar la primera fila con datos numéricos
        if header_row is None:
            for idx, row in df_raw.iterrows():
                # Verificar si la fila tiene datos numéricos
                numeric_count = sum(1 for cell in row if pd.notna(cell) and 
                                  str(cell).replace('.', '').replace('-', '').replace(':', '').isdigit())
                if numeric_count > 5:  # Si más de 5 celdas tienen datos numéricos
                    header_row = idx - 1
                    data_start_row = idx
                    break
        
        # Si aún no encontramos, usar una estrategia más agresiva
        if header_row is None:
            # Buscar la primera fila que no esté completamente vacía
            for idx, row in df_raw.iterrows():
                if not row.isna().all():
                    header_row = idx
                    data_start_row = idx + 1
                    break
        
        # Cargar el archivo con los parámetros correctos
        if file_path.suffix == '.xlsx':
            df = pd.read_excel(file_path, 
                             header=header_row if header_row is not None else 0,
                             skiprows=range(0, header_row) if header_row is not None else None,
                             nrows=max_rows,
                             engine='openpyxl')
        else:
            df = pd.read_excel(file_path, 
                             header=header_row if header_row is not None else 0,
                             skiprows=range(0, header_row) if header_row is not None else None,
                             nrows=max_rows,
                             engine='xlrd')
        
        # Limpiar nombres de columnas
        df.columns = [str(col).strip().replace('\n', ' ') if pd.notna(col) else f'Col_{i}' 
                     for i, col in enumerate(df.columns)]
        
        # Eliminar filas completamente vacías
        df = df.dropna(how='all')
        
        # Eliminar columnas completamente vacías
        df = df.dropna(axis=1, how='all')
        
        return {
            'dataframe': df,
            'filename': file_path.name,
            'shape': df.shape,
            'columns': list(df.columns),
            'dtypes': df.dtypes.to_dict(),
            'memory_usage': df.memory_usage(deep=True).sum() / (1024*1024),  # MB
            'success': True,
            'error': None,
            'header_row_found': header_row,
            'data_start_row': data_start_row
        }
    except Exception as e:
        return {
            'dataframe': None,
            'filename': file_path.name,
            'shape': (0, 0),
            'columns': [],
            'dtypes': {},
            'memory_usage': 0,
            'success': False,
            'error': str(e),
            'header_row_found': None,
            'data_start_row': None
        }

# 🔍 Examinar los primeros archivos para entender la estructura
print("📋 ANÁLISIS DE ESTRUCTURA DE ARCHIVOS DE SENSORES")
print("=" * 60)

sample_files = sensor_files_sorted[:5]  # Analizar los primeros 5 archivos
file_info = []

for i, file_path in enumerate(sample_files, 1):
    print(f"\n�� Archivo {i}: {file_path.name}")
    print("-" * 40)
    
    info = load_sensor_file(file_path, max_rows=1000)  # Cargar solo 1000 filas para análisis inicial
    file_info.append(info)
    
    if info['success']:
        df = info['dataframe']
        print(f"✅ Carga exitosa")
        print(f"�� Dimensiones: {info['shape'][0]:,} filas × {info['shape'][1]} columnas")
        print(f"💾 Uso de memoria: {info['memory_usage']:.2f} MB")
        print(f"🔢 Tipos de datos únicos: {len(set(info['dtypes'].values()))}")
        
        # Mostrar las primeras columnas
        print(f"�� Primeras 10 columnas:")
        for j, col in enumerate(info['columns'][:10], 1):
            dtype = info['dtypes'].get(col, 'unknown')
            print(f"   {j:2d}. {col:<30} ({dtype})")
        
        if len(info['columns']) > 10:
            print(f"   ... y {len(info['columns']) - 10} columnas más")
        
        # Mostrar muestra de datos
        print(f"\n�� Muestra de datos (primeras 3 filas):")
        display(df.head(3))
        
    else:
        print(f"❌ Error al cargar: {info['error']}")

print(f"\n📊 RESUMEN DE ARCHIVOS ANALIZADOS:")
print("=" * 40)
successful_loads = [info for info in file_info if info['success']]
print(f"✅ Archivos cargados exitosamente: {len(successful_loads)}/{len(file_info)}")

if successful_loads:
    total_rows = sum(info['shape'][0] for info in successful_loads)
    total_cols = [info['shape'][1] for info in successful_loads]
    total_memory = sum(info['memory_usage'] for info in successful_loads)
    
    print(f"📏 Total de filas analizadas: {total_rows:,}")
    print(f"�� Rango de columnas: {min(total_cols)} - {max(total_cols)}")
    print(f"💾 Memoria total utilizada: {total_memory:.2f} MB")

📋 ANÁLISIS DE ESTRUCTURA DE ARCHIVOS DE SENSORES

�� Archivo 1: 01-2024.xls
----------------------------------------
✅ Carga exitosa
�� Dimensiones: 744 filas × 34 columnas
💾 Uso de memoria: 0.23 MB
🔢 Tipos de datos únicos: 3
�� Primeras 10 columnas:
    1. ESTADO                         (object)
    2. Hora                           (datetime64[ns])
    3. RPM                            (float64)
    4. Presión Succión                (float64)
    5. Presión Intermedia             (float64)
    6. Presión Descarga               (float64)
    7. Pres. Aceite Comp              (float64)
    8. Temp. Cilindro # 1             (float64)
    9. Temp. Cilindro # 2             (float64)
   10. Temp. Cilindro # 3             (float64)
   ... y 24 columnas más

�� Muestra de datos (primeras 3 filas):


,ESTADO,Hora,RPM,Presión Succión,Presión Intermedia,Presión Descarga,Pres. Aceite Comp,Temp. Cilindro # 1,Temp. Cilindro # 2,Temp. Cilindro # 3,Temp. Cilindro # 4,Presión Aceite Motor,Presión Agua,Presion Mult Adm Izq,Presion Mult Adm Der,Presion Carter,Temp. Aceite Motor,Temp. Agua Motor,Temp. Mult Adm Izq,Temp. Mult Adm Der,Temp. Cil #1 L,Temp. Cil #1 R,Temp. Cil #2 L,Temp. Cil #2 R,Temp. Cil #3 L,Temp. Cil #3 R,Temp. Mult Esc Izq,Temp. Cil #4 L,Temp. Cil #4 R,Temp. Cil #5 L,Temp. Cil #5 R,Temp. Cil #6 L,Temp. Cil #6 R,Temp. Mult Esc Der
1,L,2024-01-01 00:00:00,852.637329,251.783905,492.887909,481.991913,33.750099,173.210297,157.138702,158.029404,170.499405,46.119320,13.45822,0.679329,0.296916,0.791417,184.712204,155.938095,113.142700,114.187798,913.094727,666.346313,868.581726,692.105713,713.106018,333.250488,939.555420,738.036621,691.917419,991.604980,357.024689,778.001099,834.956421,910.073914
2,L,2024-01-01 01:00:00,852.879211,251.939499,493.704407,481.920410,34.273670,172.091400,155.628296,156.286697,169.182693,46.288361,13.30897,0.670751,0.304191,0.767361,183.853394,155.047394,112.481903,112.948799,911.533081,664.185425,866.798889,690.785400,711.567322,250.003601,942.005005,735.753479,690.091980,990.005615,357.205109,777.785583,835.564087,906.461487
3,L,2024-01-01 02:00:00,852.747681,251.851807,494.205109,482.057587,34.810841,171.583801,155.268799,155.985001,168.770294,46.235199,13.01916,0.626015,0.317383,0.854756,184.169998,156.448395,112.206497,112.626503,915.816406,665.399292,869.438171,691.424683,713.851196,289.499695,940.233276,737.072693,690.842224,988.816772,356.187805,778.372986,836.507812,907.395386



�� Archivo 2: 01-2025.xls
----------------------------------------
✅ Carga exitosa
�� Dimensiones: 744 filas × 34 columnas
💾 Uso de memoria: 0.23 MB
🔢 Tipos de datos únicos: 3
�� Primeras 10 columnas:
    1. ESTADO                         (object)
    2. Hora                           (datetime64[ns])
    3. RPM                            (float64)
    4. Presión Succión                (float64)
    5. Presión Intermedia             (float64)
    6. Presión Descarga               (float64)
    7. Pres. Aceite Comp              (float64)
    8. Temp. Cilindro # 1             (float64)
    9. Temp. Cilindro # 2             (float64)
   10. Temp. Cilindro # 3             (float64)
   ... y 24 columnas más

�� Muestra de datos (primeras 3 filas):


,ESTADO,Hora,RPM,Presión Succión,Presión Intermedia,Presión Descarga,Pres. Aceite Comp,Temp. Cilindro # 1,Temp. Cilindro # 2,Temp. Cilindro # 3,Temp. Cilindro # 4,Presión Aceite Motor,Presión Agua,Presion Mult Adm Izq,Presion Mult Adm Der,Presion Carter,Temp. Aceite Motor,Temp. Agua Motor,Temp. Mult Adm Izq,Temp. Mult Adm Der,Temp. Cil #1 L,Temp. Cil #1 R,Temp. Cil #2 L,Temp. Cil #2 R,Temp. Cil #3 L,Temp. Cil #3 R,Temp. Mult Esc Izq,Temp. Cil #4 L,Temp. Cil #4 R,Temp. Cil #5 L,Temp. Cil #5 R,Temp. Cil #6 L,Temp. Cil #6 R,Temp. Mult Esc Der
1,R,2025-01-01 00:00:00,21.77582,486.163910,488.372192,1.823939,-0.049471,92.155090,97.770477,99.087189,94.711060,0.757946,2.824828,0.009513,0.316189,0.0,82.395943,93.626709,93.513008,87.810432,89.900360,91.831734,92.778801,92.012123,92.914101,88.398567,89.847366,92.823898,92.147423,92.874641,91.606232,92.372910,90.343460,83.894333
2,R,2025-01-01 01:00:00,21.75317,486.138794,488.871704,1.820363,0.024679,90.506287,96.703873,98.255531,93.219749,0.799312,2.642554,0.020371,0.313528,0.0,80.373428,92.193817,92.616783,85.865860,88.359123,89.847366,91.034599,90.117973,91.031601,85.653183,86.828377,90.884651,90.350609,91.251083,89.802277,90.749352,88.719902,81.188393
3,R,2025-01-01 02:00:00,21.77582,486.158112,488.827606,1.751220,0.035288,89.754028,96.105217,97.576843,92.445213,0.807776,2.902196,0.020371,0.313528,0.0,79.792526,90.606018,91.529579,85.130318,87.961121,88.719902,90.027771,88.855202,90.072868,85.366814,86.196999,90.027771,88.990494,90.078506,88.545151,89.630150,87.498108,81.143303



�� Archivo 3: 02-2024..xls
----------------------------------------
✅ Carga exitosa
�� Dimensiones: 696 filas × 34 columnas
💾 Uso de memoria: 0.21 MB
🔢 Tipos de datos únicos: 3
�� Primeras 10 columnas:
    1. ESTADO                         (object)
    2. Hora                           (datetime64[ns])
    3. RPM                            (float64)
    4. Presión Succión                (float64)
    5. Presión Intermedia             (float64)
    6. Presión Descarga               (float64)
    7. Pres. Aceite Comp              (float64)
    8. Temp. Cilindro # 1             (float64)
    9. Temp. Cilindro # 2             (float64)
   10. Temp. Cilindro # 3             (float64)
   ... y 24 columnas más

�� Muestra de datos (primeras 3 filas):


...


�� Archivo 4: 02-2025.xls
----------------------------------------
✅ Carga exitosa
�� Dimensiones: 672 filas × 34 columnas
💾 Uso de memoria: 0.21 MB
🔢 Tipos de datos únicos: 3
�� Primeras 10 columnas:
    1. ESTADO                         (object)
    2. Hora                           (datetime64[ns])
    3. RPM                            (float64)
    4. Presión Succión                (float64)
    5. Presión Intermedia             (float64)
    6. Presión Descarga               (float64)
    7. Pres. Aceite Comp              (float64)
    8. Temp. Cilindro # 1             (float64)
    9. Temp. Cilindro # 2             (float64)
   10. Temp. Cilindro # 3             (float64)
   ... y 24 columnas más

�� Muestra de datos (primeras 3 filas):


,ESTADO,Hora,RPM,Presión Succión,Presión Intermedia,Presión Descarga,Pres. Aceite Comp,Temp. Cilindro # 1,Temp. Cilindro # 2,Temp. Cilindro # 3,Temp. Cilindro # 4,Presión Aceite Motor,Presión Agua,Presion Mult Adm Izq,Presion Mult Adm Der,Presion Carter,Temp. Aceite Motor,Temp. Agua Motor,Temp. Mult Adm Izq,Temp. Mult Adm Der,Temp. Cil #1 L,Temp. Cil #1 R,Temp. Cil #2 L,Temp. Cil #2 R,Temp. Cil #3 L,Temp. Cil #3 R,Temp. Mult Esc Izq,Temp. Cil #4 L,Temp. Cil #4 R,Temp. Cil #5 L,Temp. Cil #5 R,Temp. Cil #6 L,Temp. Cil #6 R,Temp. Mult Esc Der
1,R,2025-02-01 00:00:00,21.767139,-0.876911,0.081073,0.724813,-0.026701,96.006470,99.493172,101.217201,94.827240,0.746502,3.185323,0.011685,0.317546,0.0,83.247917,119.806000,101.838699,93.141068,108.228897,108.292801,110.141899,109.375198,109.916397,102.069199,92.508209,109.971298,109.781097,110.953697,108.428101,109.595398,105.722198,85.517883
2,R,2025-02-01 01:00:00,21.767139,-0.971326,0.255121,0.715276,0.025751,95.175781,98.312653,99.977913,94.091431,0.769629,3.213933,0.013856,0.315700,0.0,82.473389,118.915298,100.542198,92.978943,107.345703,108.292801,110.006599,109.375198,110.006599,102.666801,91.741531,110.277199,109.871300,111.043800,108.480003,109.555603,105.406502,85.427689
3,R,2025-02-01 02:00:00,21.767139,-0.948438,0.295653,0.619907,0.031235,94.478699,97.847931,99.340851,93.316887,0.774397,3.234438,0.016354,0.317871,0.0,82.047394,118.262802,100.665703,92.492561,107.417099,107.841797,109.616501,108.834000,109.465401,102.554001,91.786629,109.590897,109.203102,110.374199,107.841797,108.924202,104.860001,84.751213



�� Archivo 5: 03-2024..xls
----------------------------------------
✅ Carga exitosa
�� Dimensiones: 744 filas × 34 columnas
💾 Uso de memoria: 0.23 MB
🔢 Tipos de datos únicos: 3
�� Primeras 10 columnas:
    1. ESTADO                         (object)
    2. Hora                           (datetime64[ns])
    3. RPM                            (float64)
    4. Presión Succión                (float64)
    5. Presión Intermedia             (float64)
    6. Presión Descarga               (float64)
    7. Pres. Aceite Comp              (float64)
    8. Temp. Cilindro # 1             (float64)
    9. Temp. Cilindro # 2             (float64)
   10. Temp. Cilindro # 3             (float64)
   ... y 24 columnas más

�� Muestra de datos (primeras 3 filas):


,ESTADO,Hora,RPM,Presión Succión,Presión Intermedia,Presión Descarga,Pres. Aceite Comp,Temp. Cilindro # 1,Temp. Cilindro # 2,Temp. Cilindro # 3,Temp. Cilindro # 4,Presión Aceite Motor,Presión Agua,Presion Mult Adm Izq,Presion Mult Adm Der,Presion Carter,Temp. Aceite Motor,Temp. Agua Motor,Temp. Mult Adm Izq,Temp. Mult Adm Der,Temp. Cil #1 L,Temp. Cil #1 R,Temp. Cil #2 L,Temp. Cil #2 R,Temp. Cil #3 L,Temp. Cil #3 R,Temp. Mult Esc Izq,Temp. Cil #4 L,Temp. Cil #4 R,Temp. Cil #5 L,Temp. Cil #5 R,Temp. Cil #6 L,Temp. Cil #6 R,Temp. Mult Esc Der
1,L,2024-03-01 00:00:00,859.207397,250.742401,492.871185,481.855988,31.050440,174.651596,158.542496,159.009903,172.020798,46.01263,13.33233,-0.044449,-0.134135,0.963392,181.613998,157.680801,115.299599,113.752800,899.675903,672.890198,857.721985,697.404480,689.547485,538.186707,912.270630,687.636780,702.662170,952.750488,450.578308,753.941772,831.808777,886.343201
2,L,2024-03-01 01:00:00,857.786072,251.057205,492.976013,482.552185,30.865419,174.302994,157.419495,157.848099,171.594803,45.68837,13.21836,-0.204167,-0.219259,0.933995,181.846405,157.404007,114.284401,112.931900,899.584229,673.821472,857.872986,697.754028,689.680481,542.038818,909.663818,688.078674,702.251892,952.927185,452.040314,754.271118,831.862976,887.966675
3,L,2024-03-01 02:00:00,858.504395,251.070694,492.964111,482.919403,31.041620,173.025101,156.160797,156.376404,170.239304,45.55032,13.24960,-0.081910,-0.148793,1.002088,182.117493,160.037399,113.205002,112.310204,899.069275,675.093323,856.471191,698.783691,690.794373,543.133179,911.323425,689.809692,704.091919,954.020813,450.518188,754.150879,828.869080,886.854980



📊 RESUMEN DE ARCHIVOS ANALIZADOS:
✅ Archivos cargados exitosamente: 5/5
📏 Total de filas analizadas: 3,600
�� Rango de columnas: 34 - 34
💾 Memoria total utilizada: 1.11 MB


### 📋 Carga del Historial de Fallas

El historial de fallas es crucial para nuestro análisis, ya que nos permitirá correlacionar los eventos de mantenimiento con las lecturas de sensores precedentes.

In [47]:
# 📋 Cargar historial de fallas con manejo de estructura compleja
print("📋 ANÁLISIS DEL HISTORIAL DE FALLAS")
print("=" * 50)

historial_df = None
if historial_files:
    historial_file = historial_files[0]  # Tomar el primer archivo de historial
    print(f"📁 Cargando: {historial_file.name}")
    
    try:
        # Cargar el archivo completo para analizar su estructura
        df_raw = pd.read_excel(historial_file, header=None, engine='openpyxl')
        
        # Buscar la fila donde comienzan los datos reales
        # Buscar filas que contengan indicadores de encabezados
        header_row = None
        data_start_row = None
        
        for idx, row in df_raw.iterrows():
            row_str = ' '.join(str(cell) for cell in row if pd.notna(cell))
            if 'NUMERO AVISO' in row_str.upper() or 'N°' in row_str:
                header_row = idx
                data_start_row = idx + 1
                break
        
        # Si no encontramos indicadores específicos, buscar la primera fila con datos
        if header_row is None:
            for idx, row in df_raw.iterrows():
                if not row.isna().all() and any(str(cell).isdigit() for cell in row if pd.notna(cell)):
                    header_row = idx - 1
                    data_start_row = idx
                    break
        
        # Cargar el archivo con los parámetros correctos
        historial_df = pd.read_excel(historial_file, 
                                   header=header_row if header_row is not None else 0,
                                   skiprows=range(0, header_row) if header_row is not None else None,
                                   engine='openpyxl')
        
        # Limpiar nombres de columnas
        historial_df.columns = [str(col).strip().replace('\n', ' ') if pd.notna(col) else f'Col_{i}' 
                               for i, col in enumerate(historial_df.columns)]
        
        # Eliminar filas completamente vacías
        historial_df = historial_df.dropna(how='all')
        
        # Eliminar columnas completamente vacías
        historial_df = historial_df.dropna(axis=1, how='all')
        
        print(f"✅ Historial cargado exitosamente")
        print(f"�� Dimensiones: {historial_df.shape[0]:,} eventos × {historial_df.shape[1]} campos")
        print(f"💾 Uso de memoria: {historial_df.memory_usage(deep=True).sum() / (1024*1024):.2f} MB")
        
        print(f"\n📊 Estructura del Historial de Fallas:")
        print("-" * 40)
        for i, col in enumerate(historial_df.columns, 1):
            dtype = historial_df[col].dtype
            non_null = historial_df[col].notna().sum()
            null_pct = (historial_df[col].isna().sum() / len(historial_df)) * 100
            # Corregir el formato del tipo de dato
            dtype_str = str(dtype) if hasattr(dtype, '__str__') else 'unknown'
            print(f"{i:2d}. {col:<25} | {dtype_str:<12} | {non_null:>6} valores | {null_pct:5.1f}% nulos")
        
        print(f"\n👀 Muestra del Historial (primeras 5 filas):")
        display(historial_df.head())
        
        print(f"\n�� Últimas 5 entradas del historial:")
        display(historial_df.tail())
        
        # Análisis básico de fechas si existe una columna de fecha
        date_columns = [col for col in historial_df.columns if any(keyword in col.lower() for keyword in ['fecha', 'date', 'tiempo', 'time', 'creado'])]
        if date_columns:
            print(f"\n📅 Análisis Temporal del Historial:")
            print("-" * 30)
            for date_col in date_columns[:2]:  # Analizar máximo 2 columnas de fecha
                print(f"\n�� Columna: {date_col}")
                try:
                    # Intentar convertir a datetime
                    date_series = pd.to_datetime(historial_df[date_col], errors='coerce')
                    valid_dates = date_series.dropna()
                    
                    if len(valid_dates) > 0:
                        print(f"   ✅ Fechas válidas: {len(valid_dates)}/{len(historial_df)}")
                        print(f"   📅 Rango: {valid_dates.min()} a {valid_dates.max()}")
                        print(f"   📊 Período total: {(valid_dates.max() - valid_dates.min()).days} días")
                    else:
                        print(f"   ❌ No se pudieron interpretar fechas válidas")
                except Exception as e:
                    print(f"   ❌ Error procesando fechas: {e}")
        
        # Análisis de tipos de eventos si existe una columna descriptiva
        text_columns = historial_df.select_dtypes(include=['object']).columns
        if len(text_columns) > 0:
            print(f"\n🔍 Análisis de Tipos de Eventos:")
            print("-" * 35)
            
            for col in text_columns[:3]:  # Analizar máximo 3 columnas de texto
                if historial_df[col].notna().sum() > 0:
                    print(f"\n📋 Columna: {col}")
                    value_counts = historial_df[col].value_counts().head(10)
                    print(f"   🔢 Valores únicos: {historial_df[col].nunique()}")
                    print(f"   📊 Top 5 más frecuentes:")
                    for i, (value, count) in enumerate(value_counts.head(5).items(), 1):
                        pct = (count / len(historial_df)) * 100
                        print(f"      {i}. {str(value)[:50]:<52} | {count:>3} ({pct:4.1f}%)")
        
    except Exception as e:
        print(f"❌ Error cargando historial: {e}")
        historial_df = None
else:
    print("⚠️ No se encontró archivo de historial de fallas")

📋 ANÁLISIS DEL HISTORIAL DE FALLAS
📁 Cargando: Historial C1 RGD.xlsx


✅ Historial cargado exitosamente
�� Dimensiones: 85 eventos × 17 campos
💾 Uso de memoria: 0.13 MB

📊 Estructura del Historial de Fallas:
----------------------------------------
 1. 3                         | int64        |     85 valores |   0.0% nulos
 2. 200228352                 | int64        |     85 valores |   0.0% nulos
 3. B2                        | object       |     85 valores |   0.0% nulos
 4. Reempalzo codos de escape #2 | object       |     85 valores |   0.0% nulos
 5. 2023-01-12 00:00:00       | object       |     85 valores |   0.0% nulos
 6. 2023-01-12 00:00:00.1     | object       |     85 valores |   0.0% nulos
 7. 2023-01-12 00:00:00.2     | object       |     78 valores |   8.2% nulos
 8. 09:00:00                  | object       |     65 valores |  23.5% nulos
 9. 17:45:00                  | object       |     65 valores |  23.5% nulos
10. 8.75                      | float64      |     85 valores |   0.0% nulos
11. 2000245544                | int64        |   

,3,200228352,B2,Reempalzo codos de escape #2,2023-01-12 00:00:00,2023-01-12 00:00:00.1,2023-01-12 00:00:00.2,09:00:00,17:45:00,8.75,2000245544,PB01,MECANICO,RB001793,RB-RGD-RGDI-PTA-COM-C01,DGOMEZ,"* 12/01/2023 15:45:33 UTC-4 DANIEL WILSON GOMEZ TRINCADO (DGOMEZ) * 11/01/2023 * Unidad en reserva, se sacaron las bujías de ambos bancos, se observó agua en el cilindro de potencia #4 R, se sacó tapa de inspección de codos de escape, se abrieron las válvulas, se observa pinchadura del codo de escape #2, Se hicieron prueba hidráulica de 2 codos de escap taller INY/ se pasó macho de 3/8"" y de 1/2"" a la rosca y se lijo de horas 14:30 a 18:00 O/ Acosta, X/ Zurita, J/ Valdelomar/ * 13/01/2023 10:27:56 UTC-4 DANIEL WILSON GOMEZ TRINCADO (DGOMEZ) * 12/01/2023 * Unidad en reserva, se elaboró ATS/ Se drenó el agua refrigerante del motor de sacaron codos de agua, pernos y codo de escape #1, #2, #3, R/ que tenía pinchadura, se realizó limpieza a los codos de escape, de agua, se cambiaron empaquetaduras y oring's nuevos, se colocó cod escape en condición B/ con empaquetaduras nueva y los otros codos lo mismo con empaquetadura nueva sacadas del almacén, se ajustaron los pernos, se colocaron los codos de agua, se presurizó el sistema, se eliminó perdida, se sopletearon los cilindros de potencia sin agua, colocaron sus misma bujías cepilladas y calibradas, se dio secuencia arranque y se hizo funcionar la unidad en vació por espació de 15 mi luego se paró a reserva a horas 18:00 de horas 09:00 a 11:30 y de 13 a 18:00 O/ Acosta, X/ Zurita, J/ Valdelomar/ Se sacó termocupla de temperatura de agua por trabajos mecánicos en codos, se sacó de almacen un niple hexagonal de 3/4 para Ventrap, por encontrarse niple de 3/4 roto, de horas 16:00 a 18:00 (R/ Callau, C/ Espinoza)"
0,4,200228552,B2,"Cambio manguera de 1 3/8"" bba aux",2023-01-18 00:00:00,2023-01-17 00:00:00,2023-01-17 00:00:00,09:00:00,15:00:00,6.0,2000245856,PB01,MECANICO,RB001793,RB-RGD-RGDI-PTA-COM-C01,DGOMEZ,18/01/2023 10:38:56 UTC-4 DANIEL WILSON GOMEZ ...
1,5,200228594,B1,Muestreo Aceite COMP AMA4 C#1 INY(500),2023-01-19 00:00:00,2023-01-19 00:00:00,2023-01-19 00:00:00,NaN,NaN,0.0,2000244478,PB07,MECANICO,RB001792,RB-RGD-RGDI-PTA-COM-C01,DGOMEZ,* 19/01/2023 10:09:23 UTC-4 DANIEL WILSON GOME...
2,6,200228595,B1,Muestreo Aceite MEXPWK7042 C#1 INY(500),2023-01-19 00:00:00,2023-01-19 00:00:00,2023-01-19 00:00:00,NaN,NaN,0.0,2000244479,PB07,MECANICO,RB001793,RB-RGD-RGDI-PTA-COM-C01,DGOMEZ,* 19/01/2023 10:10:21 UTC-4 DANIEL WILSON GOME...
3,7,200228600,B2,Limpieza valvula de 3/4 del ventrap,2023-01-19 00:00:00,2023-01-18 00:00:00,2023-01-18 00:00:00,09:00:00,10:30:00,1.5,2000245872,PB01,INSTRUME,RB001793,RB-RGD-RGDI-PTA-COM-C01,DGOMEZ,* 19/01/2023 10:40:32 UTC-4 DANIEL WILSON GOME...
4,8,200229408,B2,Regulacion golpeto caja markord,2023-02-13 00:00:00,2023-02-13 00:00:00,2023-02-13 00:00:00,10:30:00,11:30:00,1.0,2000246957,PB01,MECANICO,RB001793,RB-RGD-RGDI-PTA-COM-C01,DGOMEZ,* 13/02/2023 11:01:01 UTC-4 DANIEL WILSON GOME...



�� Últimas 5 entradas del historial:


,3,200228352,B2,Reempalzo codos de escape #2,2023-01-12 00:00:00,2023-01-12 00:00:00.1,2023-01-12 00:00:00.2,09:00:00,17:45:00,8.75,2000245544,PB01,MECANICO,RB001793,RB-RGD-RGDI-PTA-COM-C01,DGOMEZ,"* 12/01/2023 15:45:33 UTC-4 DANIEL WILSON GOMEZ TRINCADO (DGOMEZ) * 11/01/2023 * Unidad en reserva, se sacaron las bujías de ambos bancos, se observó agua en el cilindro de potencia #4 R, se sacó tapa de inspección de codos de escape, se abrieron las válvulas, se observa pinchadura del codo de escape #2, Se hicieron prueba hidráulica de 2 codos de escap taller INY/ se pasó macho de 3/8"" y de 1/2"" a la rosca y se lijo de horas 14:30 a 18:00 O/ Acosta, X/ Zurita, J/ Valdelomar/ * 13/01/2023 10:27:56 UTC-4 DANIEL WILSON GOMEZ TRINCADO (DGOMEZ) * 12/01/2023 * Unidad en reserva, se elaboró ATS/ Se drenó el agua refrigerante del motor de sacaron codos de agua, pernos y codo de escape #1, #2, #3, R/ que tenía pinchadura, se realizó limpieza a los codos de escape, de agua, se cambiaron empaquetaduras y oring's nuevos, se colocó cod escape en condición B/ con empaquetaduras nueva y los otros codos lo mismo con empaquetadura nueva sacadas del almacén, se ajustaron los pernos, se colocaron los codos de agua, se presurizó el sistema, se eliminó perdida, se sopletearon los cilindros de potencia sin agua, colocaron sus misma bujías cepilladas y calibradas, se dio secuencia arranque y se hizo funcionar la unidad en vació por espació de 15 mi luego se paró a reserva a horas 18:00 de horas 09:00 a 11:30 y de 13 a 18:00 O/ Acosta, X/ Zurita, J/ Valdelomar/ Se sacó termocupla de temperatura de agua por trabajos mecánicos en codos, se sacó de almacen un niple hexagonal de 3/4 para Ventrap, por encontrarse niple de 3/4 roto, de horas 16:00 a 18:00 (R/ Callau, C/ Espinoza)"
80,84,200253559,B2,Reemplazo de bujias,2024-10-18 00:00:00,2024-10-14 00:00:00,2024-10-14 00:00:00,18:00:00,18:15:00,0.25,2000273603,PB01,MECANICO,RB001793,RB-RGD-RGDI-PTA-COM-C01,DGOMEZ,* 18/10/2024 15:58:39 UTC-4 DANIEL WILSON GOME...
81,85,200259011,B3,Análisis Vibraciones RB001792(Exterran),2025-02-19 00:00:00,01.07.2025,NaN,00:00:00,00:00:00,0.00,2000279289,PB06,MANTPRED,RB001792,RB-RGD-RGDI-PTA-COM-C01,BO00260,01/07/2025 10:40:12 UTC-4 JOHANNA ORDOÑEZ GUER...
82,86,200259012,B3,Análisis Vibraciones RB001793(Exterran),2025-02-19 00:00:00,01.07.2025,NaN,00:00:00,00:00:00,0.00,2000279290,PB06,MANTPRED,RB001793,RB-RGD-RGDI-PTA-COM-C01,BO00260,01/07/2025 10:42:34 UTC-4 JOHANNA ORDOÑEZ GUER...
83,87,200262983,B1,Cabmio de ubicacion vlv señal de presion,2025-05-21 00:00:00,20.05.2025,20.05.2025,00:00:00,00:00:00,0.00,2000283968,PB04,INSTRUME,RB001423,RB-RGD-RGDI-PTA-COM-C01,DGOMEZ,21/05/2025 10:59:00 UTC-4 DANIEL WILSON GOMEZ ...
84,88,200263214,B8,OVH Compresor AMA-4 C-01 INY(HB),2025-04-11 00:00:00,2025-04-11 00:00:00,NaN,07:00:00,00:00:00,0.00,2000221067,PB01,OVH-TOP,RB001792,RB-RGD-RGDI-PTA-COM-C01,BO00260,Continua en ejecucion



🔍 Análisis de Tipos de Eventos:
-----------------------------------

📋 Columna: B2
   🔢 Valores únicos: 5
   📊 Top 5 más frecuentes:
      1. B2                                                   |  40 (47.1%)
      2. B1                                                   |  22 (25.9%)
      3. B3                                                   |  21 (24.7%)
      4. B3
                                                  |   1 ( 1.2%)
      5. B8                                                   |   1 ( 1.2%)

📋 Columna: Reempalzo codos de escape #2
   🔢 Valores únicos: 55
   📊 Top 5 más frecuentes:
      1. Muestreo Aceite MEXPWK7042 C#1 INY(500)              |  10 (11.8%)
      2. Muestreo Aceite COMP AMA4 C#1 INY(500)               |   9 (10.6%)
      3. Mant/ MEXP WK L-7042 de C-01 INY                     |   5 ( 5.9%)
      4. Mant/ Compresor Gas Baja C-01 INY                    |   5 ( 5.9%)
      5. Mant/ Tablero Control de C-01 INY                    |   4 ( 4.7%)

📋 Columna: 20

### 🔗 Consolidación de Datos de Sensores

Para realizar un análisis comprehensivo, necesitamos consolidar los datos de múltiples archivos en un dataset unificado. Esto nos permitirá tener una visión temporal completa del comportamiento del motocompresor.

In [48]:
# 🔗 Consolidación inteligente de datos de sensores
print("🔗 CONSOLIDACIÓN DE DATOS DE SENSORES")
print("=" * 50)

def smart_load_and_combine_sensors(file_list, max_files=10, sample_size=None):
    """
    Carga y combina múltiples archivos de sensores de manera inteligente
    """
    combined_data = []
    successful_files = []
    failed_files = []
    
    print(f"📊 Procesando {min(len(file_list), max_files)} archivos de {len(file_list)} disponibles...")
    
    for i, file_path in enumerate(file_list[:max_files], 1):
        print(f"\n📁 [{i:2d}/{min(len(file_list), max_files)}] Procesando: {file_path.name}")
        
        try:
            # Usar la función mejorada de carga
            if file_path.suffix == '.xlsx':
                df_raw = pd.read_excel(file_path, header=None, engine='openpyxl')
            else:
                df_raw = pd.read_excel(file_path, header=None, engine='xlrd')
            
            # Buscar la fila donde comienzan los datos reales
            header_row = None
            data_start_row = None
            
            for idx, row in df_raw.iterrows():
                row_str = ' '.join(str(cell) for cell in row if pd.notna(cell))
                if 'COMPRESOR' in row_str.upper() or 'MOTOR' in row_str.upper():
                    header_row = idx
                    data_start_row = idx + 1
                    break
            
            # Si no encontramos indicadores específicos, buscar la primera fila con datos numéricos
            if header_row is None:
                for idx, row in df_raw.iterrows():
                    numeric_count = sum(1 for cell in row if pd.notna(cell) and 
                                      str(cell).replace('.', '').replace('-', '').replace(':', '').isdigit())
                    if numeric_count > 5:
                        header_row = idx - 1
                        data_start_row = idx
                        break
            
            # Si aún no encontramos, usar una estrategia más agresiva
            if header_row is None:
                for idx, row in df_raw.iterrows():
                    if not row.isna().all():
                        header_row = idx
                        data_start_row = idx + 1
                        break
            
            # Cargar el archivo con los parámetros correctos
            if file_path.suffix == '.xlsx':
                df = pd.read_excel(file_path, 
                                 header=header_row if header_row is not None else 0,
                                 skiprows=range(0, header_row) if header_row is not None else None,
                                 nrows=sample_size,
                                 engine='openpyxl')
            else:
                df = pd.read_excel(file_path, 
                                 header=header_row if header_row is not None else 0,
                                 skiprows=range(0, header_row) if header_row is not None else None,
                                 nrows=sample_size,
                                 engine='xlrd')
            
            # Limpiar nombres de columnas
            df.columns = [str(col).strip().replace('\n', ' ') if pd.notna(col) else f'Col_{i}' 
                         for i, col in enumerate(df.columns)]
            
            # Eliminar filas completamente vacías
            df = df.dropna(how='all')
            
            # Eliminar columnas completamente vacías
            df = df.dropna(axis=1, how='all')
            
            # Agregar información del archivo
            df['archivo_origen'] = file_path.name
            df['indice_archivo'] = i
            
            # Intentar extraer fecha del nombre del archivo
            filename = file_path.stem
            
            # Buscar patrones de fecha en el nombre del archivo
            import re
            date_patterns = [
                r'(\d{4}-\d{2}-\d{2})',  # YYYY-MM-DD
                r'(\d{2}-\d{4})',        # MM-YYYY
                r'(\d{1,2}-\d{4})',      # M-YYYY
            ]
            
            fecha_archivo = None
            for pattern in date_patterns:
                match = re.search(pattern, filename)
                if match:
                    try:
                        fecha_str = match.group(1)
                        if len(fecha_str.split('-')) == 2:  # MM-YYYY format
                            month, year = fecha_str.split('-')
                            fecha_archivo = pd.to_datetime(f"{year}-{month.zfill(2)}-01")
                        else:
                            fecha_archivo = pd.to_datetime(fecha_str)
                        break
                    except:
                        continue
            
            if fecha_archivo:
                df['fecha_archivo'] = fecha_archivo
                print(f"   �� Fecha extraída: {fecha_archivo.strftime('%Y-%m-%d')}")
            else:
                df['fecha_archivo'] = pd.NaT
                print(f"   ⚠️ No se pudo extraer fecha del nombre")
            
            print(f"   ✅ Cargado: {df.shape[0]:,} filas × {df.shape[1]} columnas")
            
            combined_data.append(df)
            successful_files.append(file_path.name)
            
        except Exception as e:
            print(f"   ❌ Error: {str(e)[:100]}...")
            failed_files.append((file_path.name, str(e)))
    
    # Combinar todos los dataframes
    if combined_data:
        print(f"\n🔗 Combinando {len(combined_data)} archivos exitosos...")
        
        # Encontrar columnas comunes
        all_columns = [set(df.columns) for df in combined_data]
        common_columns = set.intersection(*all_columns) if all_columns else set()
        
        print(f"   📊 Columnas comunes encontradas: {len(common_columns)}")
        
        if len(common_columns) > 5:  # Solo combinar si hay suficientes columnas comunes
            # Seleccionar solo columnas comunes para la combinación
            standardized_data = []
            for df in combined_data:
                # Seleccionar columnas comunes más las metadatos que agregamos
                cols_to_keep = list(common_columns) + ['archivo_origen', 'indice_archivo', 'fecha_archivo']
                cols_to_keep = [col for col in cols_to_keep if col in df.columns]
                standardized_data.append(df[cols_to_keep])
            
            combined_df = pd.concat(standardized_data, ignore_index=True, sort=False)
            
            print(f"   ✅ Dataset combinado: {combined_df.shape[0]:,} filas × {combined_df.shape[1]} columnas")
            
            return {
                'dataframe': combined_df,
                'successful_files': successful_files,
                'failed_files': failed_files,
                'common_columns': list(common_columns),
                'total_files_processed': len(combined_data)
            }
        else:
            print(f"   ⚠️ Muy pocas columnas comunes ({len(common_columns)}). Retornando primer archivo.")
            return {
                'dataframe': combined_data[0],
                'successful_files': successful_files[:1],
                'failed_files': failed_files,
                'common_columns': list(combined_data[0].columns),
                'total_files_processed': 1
            }
    else:
        print(f"   ❌ No se pudo cargar ningún archivo exitosamente")
        return None

# Ejecutar consolidación
consolidation_result = smart_load_and_combine_sensors(
    sensor_files_sorted, 
    max_files=15,  # Procesar máximo 15 archivos para mantener rendimiento
    sample_size=5000  # Máximo 5000 filas por archivo
)

if consolidation_result:
    sensors_df = consolidation_result['dataframe']
    
    print(f"\n📊 RESULTADO DE LA CONSOLIDACIÓN:")
    print("=" * 45)
    print(f"✅ Archivos procesados exitosamente: {len(consolidation_result['successful_files'])}")
    print(f"❌ Archivos con errores: {len(consolidation_result['failed_files'])}")
    print(f"📏 Dataset final: {sensors_df.shape[0]:,} filas × {sensors_df.shape[1]} columnas")
    print(f"�� Uso de memoria: {sensors_df.memory_usage(deep=True).sum() / (1024*1024):.2f} MB")
    
    if consolidation_result['failed_files']:
        print(f"\n❌ Archivos con errores:")
        for filename, error in consolidation_result['failed_files'][:5]:
            print(f"   • {filename}: {error[:80]}...")
    
    # Mostrar información básica del dataset consolidado
    print(f"\n📋 Información General del Dataset Consolidado:")
    print("-" * 50)
    print(sensors_df.info(memory_usage='deep'))
    
else:
    print("❌ No se pudo consolidar los datos de sensores")
    sensors_df = None

🔗 CONSOLIDACIÓN DE DATOS DE SENSORES
📊 Procesando 15 archivos de 28 disponibles...

📁 [ 1/15] Procesando: 01-2024.xls
   �� Fecha extraída: 2024-01-01
   ✅ Cargado: 744 filas × 37 columnas

📁 [ 2/15] Procesando: 01-2025.xls
   �� Fecha extraída: 2025-01-01
   ✅ Cargado: 744 filas × 37 columnas

📁 [ 3/15] Procesando: 02-2024..xls
   �� Fecha extraída: 2024-02-01
   ✅ Cargado: 696 filas × 37 columnas

📁 [ 4/15] Procesando: 02-2025.xls
   �� Fecha extraída: 2025-02-01
   ✅ Cargado: 672 filas × 37 columnas

📁 [ 5/15] Procesando: 03-2024..xls
   �� Fecha extraída: 2024-03-01
   ✅ Cargado: 744 filas × 37 columnas

📁 [ 6/15] Procesando: 03-2025.xls
   �� Fecha extraída: 2025-03-01
   ✅ Cargado: 744 filas × 37 columnas

📁 [ 7/15] Procesando: 04-2024..xls
   �� Fecha extraída: 2024-04-01
   ✅ Cargado: 720 filas × 37 columnas

📁 [ 8/15] Procesando: 04-2025.xls
   �� Fecha extraída: 2025-04-01
   ✅ Cargado: 720 filas × 37 columnas

📁 [ 9/15] Procesando: 05-2024..xls
   �� Fecha extraída: 2024-05-

In [30]:
sensors_df.head(2)

,Unnamed: 12,Unnamed: 7,Unnamed: 8,archivo_origen,Unnamed: 4,Unnamed: 23,Unnamed: 5,indice_archivo,Unnamed: 14,Unnamed: 10,Unnamed: 6,Unnamed: 3,Unnamed: 13,Unnamed: 17,Unnamed: 19,Unnamed: 30,Unnamed: 26,Unnamed: 2,Unnamed: 29,Unnamed: 32,Unnamed: 11,Unnamed: 0,fecha_archivo,Unnamed: 22,Unnamed: 20,Unnamed: 16,Unnamed: 24,Unnamed: 9,Unnamed: 1,Unnamed: 31,Unnamed: 21,Unnamed: 15,Unnamed: 33,Unnamed: 18,Unnamed: 27,Unnamed: 25,Unnamed: 28,archivo_origen,indice_archivo,fecha_archivo
0,NaN,NaN,NaN,01-2024.xls,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MES,2024-01-01,NaN,NaN,NaN,NaN,NaN,2024-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01-2024.xls,1,2024-01-01
1,NaN,NaN,NaN,01-2024.xls,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COMPRESOR # 1,NaN,NaN,NaN,NaN,NaN,01-2024.xls,1,2024-01-01


---

## 2. 🧹 Limpieza y Validación de Datos

La calidad de los datos es fundamental para el éxito de cualquier modelo de mantenimiento predictivo. En esta sección realizaremos una limpieza exhaustiva y validación de los datos para asegurar su integridad y confiabilidad.

### 🔍 Análisis de Valores Faltantes

In [ ]:
# 🔍 Análisis detallado de valores faltantes
if sensors_df is not None:
    print("🔍 ANÁLISIS DE VALORES FALTANTES")
    print("=" * 50)
    
    # Calcular estadísticas de valores faltantes
    missing_stats = pd.DataFrame({
        'Columna': sensors_df.columns,
        'Valores_Faltantes': sensors_df.isnull().sum(),
        'Porcentaje_Faltante': (sensors_df.isnull().sum() / len(sensors_df)) * 100,
        'Tipo_Dato': sensors_df.dtypes,
        'Valores_Únicos': [sensors_df[col].nunique() for col in sensors_df.columns]
    })
    
    missing_stats = missing_stats.sort_values('Porcentaje_Faltante', ascending=False)
    
    print(f"📊 Total de columnas: {len(sensors_df.columns)}")
    print(f"📊 Total de filas: {len(sensors_df):,}")
    
    # Columnas con valores faltantes
    columns_with_missing = missing_stats[missing_stats['Valores_Faltantes'] > 0]
    
    if len(columns_with_missing) > 0:
        print(f"\n⚠️ Columnas con valores faltantes: {len(columns_with_missing)}")
        print("-" * 80)
        print(f"{'Columna':<25} | {'Faltantes':>10} | {'%':>6} | {'Tipo':<12} | {'Únicos':>8}")
        print("-" * 80)
        
        for _, row in columns_with_missing.head(20).iterrows():
            col_name = str(row['Columna'])[:24]
            print(f"{col_name:<25} | {row['Valores_Faltantes']:>10,} | {row['Porcentaje_Faltante']:>5.1f}% | {str(row['Tipo_Dato']):<12} | {row['Valores_Únicos']:>8,}")
        
        if len(columns_with_missing) > 20:
            print(f"... y {len(columns_with_missing) - 20} columnas más con valores faltantes")
    else:
        print("✅ No se encontraron valores faltantes en el dataset")
    
    # Visualización de patrones de valores faltantes
    if len(columns_with_missing) > 0:
        print(f"\n📊 Visualización de Patrones de Valores Faltantes:")
        
        # Seleccionar top 15 columnas con más valores faltantes para visualización
        top_missing_cols = columns_with_missing.head(15)['Columna'].tolist()
        
        if len(top_missing_cols) > 0:
            plt.figure(figsize=(14, 8))
            
            # Subplot 1: Gráfico de barras de valores faltantes
            plt.subplot(2, 1, 1)
            missing_counts = columns_with_missing.head(15)
            bars = plt.bar(range(len(missing_counts)), missing_counts['Porcentaje_Faltante'])
            plt.title('🔍 Porcentaje de Valores Faltantes por Columna (Top 15)', fontsize=14, fontweight='bold')
            plt.xlabel('Columnas')
            plt.ylabel('% Valores Faltantes')
            plt.xticks(range(len(missing_counts)), 
                      [col[:20] + '...' if len(col) > 20 else col for col in missing_counts['Columna']], 
                      rotation=45, ha='right')
            
            # Colorear barras según severidad
            for i, bar in enumerate(bars):
                pct = missing_counts.iloc[i]['Porcentaje_Faltante']
                if pct > 50:
                    bar.set_color('red')
                elif pct > 20:
                    bar.set_color('orange')
                else:
                    bar.set_color('yellow')
            
            plt.grid(axis='y', alpha=0.3)
            
            # Subplot 2: Heatmap de patrones de valores faltantes
            plt.subplot(2, 1, 2)
            
            # Tomar muestra para heatmap si hay demasiadas filas
            sample_size = min(1000, len(sensors_df))
            sample_indices = np.random.choice(len(sensors_df), sample_size, replace=False)
            sample_df = sensors_df.iloc[sample_indices][top_missing_cols[:10]]  # Top 10 para visualización
            
            # Crear matriz de valores faltantes (1 = faltante, 0 = presente)
            missing_matrix = sample_df.isnull().astype(int)
            
            sns.heatmap(missing_matrix.T, cbar=True, cmap='RdYlBu_r', 
                       yticklabels=[col[:20] + '...' if len(col) > 20 else col for col in missing_matrix.columns],
                       xticklabels=False)
            plt.title(f'🔍 Patrón de Valores Faltantes (Muestra de {sample_size:,} filas)', fontsize=14, fontweight='bold')
            plt.xlabel('Observaciones (muestra)')
            plt.ylabel('Variables')
            
            plt.tight_layout()
            plt.show()
    
    # Categorización de columnas por severidad de valores faltantes
    print(f"\n📊 Categorización por Severidad:")
    print("-" * 40)
    
    criticas = missing_stats[missing_stats['Porcentaje_Faltante'] > 50]
    moderadas = missing_stats[(missing_stats['Porcentaje_Faltante'] > 20) & (missing_stats['Porcentaje_Faltante'] <= 50)]
    leves = missing_stats[(missing_stats['Porcentaje_Faltante'] > 0) & (missing_stats['Porcentaje_Faltante'] <= 20)]
    completas = missing_stats[missing_stats['Porcentaje_Faltante'] == 0]
    
    print(f"🔴 CRÍTICAS (>50% faltantes): {len(criticas)} columnas")
    print(f"🟠 MODERADAS (20-50% faltantes): {len(moderadas)} columnas")
    print(f"🟡 LEVES (<20% faltantes): {len(leves)} columnas")
    print(f"🟢 COMPLETAS (sin faltantes): {len(completas)} columnas")
    
else:
    print("❌ No hay datos de sensores disponibles para análisis de valores faltantes")

### 🔧 Estrategia de Limpieza de Datos

Basándose en el análisis anterior, implementaremos una estrategia de limpieza específica para optimizar la calidad de los datos para el mantenimiento predictivo.

In [ ]:
# 🔧 Implementación de estrategia de limpieza
if sensors_df is not None:
    print("🔧 ESTRATEGIA DE LIMPIEZA DE DATOS")
    print("=" * 50)
    
    # Hacer copia para limpieza
    sensors_clean = sensors_df.copy()
    print(f"📊 Dataset original: {sensors_clean.shape[0]:,} filas × {sensors_clean.shape[1]} columnas")
    
    # 1. Eliminar columnas con más del 80% de valores faltantes
    print(f"\n🗑️ Paso 1: Eliminando columnas con >80% valores faltantes...")
    
    high_missing_cols = missing_stats[missing_stats['Porcentaje_Faltante'] > 80]['Columna'].tolist()
    
    if high_missing_cols:
        print(f"   Columnas a eliminar: {len(high_missing_cols)}")
        for col in high_missing_cols[:10]:  # Mostrar solo las primeras 10
            pct = missing_stats[missing_stats['Columna'] == col]['Porcentaje_Faltante'].iloc[0]
            print(f"   • {col[:40]:<40} ({pct:.1f}% faltante)")
        if len(high_missing_cols) > 10:
            print(f"   ... y {len(high_missing_cols) - 10} más")
        
        sensors_clean = sensors_clean.drop(columns=high_missing_cols)
        print(f"   ✅ Columnas eliminadas: {len(high_missing_cols)}")
    else:
        print(f"   ✅ No hay columnas con >80% valores faltantes")
    
    print(f"   📊 Dataset después del paso 1: {sensors_clean.shape[0]:,} filas × {sensors_clean.shape[1]} columnas")
    
    # 2. Identificar y manejar duplicados
    print(f"\n🔍 Paso 2: Identificando filas duplicadas...")
    
    # Excluir columnas de metadatos para detección de duplicados
    metadata_cols = ['archivo_origen', 'indice_archivo', 'fecha_archivo']
    data_cols = [col for col in sensors_clean.columns if col not in metadata_cols]
    
    if data_cols:
        duplicates = sensors_clean.duplicated(subset=data_cols)
        num_duplicates = duplicates.sum()
        
        if num_duplicates > 0:
            print(f"   ⚠️ Filas duplicadas encontradas: {num_duplicates:,} ({(num_duplicates/len(sensors_clean))*100:.2f}%)")
            sensors_clean = sensors_clean[~duplicates]
            print(f"   ✅ Filas duplicadas eliminadas")
        else:
            print(f"   ✅ No se encontraron filas duplicadas")
    
    print(f"   📊 Dataset después del paso 2: {sensors_clean.shape[0]:,} filas × {sensors_clean.shape[1]} columnas")
    
    # 3. Identificar columnas numéricas para validación de rangos
    print(f"\n🔢 Paso 3: Validando rangos de datos numéricos...")
    
    numeric_cols = sensors_clean.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols = [col for col in numeric_cols if col not in metadata_cols + ['indice_archivo']]
    
    print(f"   📊 Columnas numéricas para validar: {len(numeric_cols)}")
    
    outlier_summary = []
    
    if numeric_cols:
        for col in numeric_cols[:15]:  # Analizar las primeras 15 columnas numéricas
            if sensors_clean[col].notna().sum() > 0:  # Solo si hay datos válidos
                # Calcular estadísticas
                Q1 = sensors_clean[col].quantile(0.25)
                Q3 = sensors_clean[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                
                # Identificar outliers
                outliers = (sensors_clean[col] < lower_bound) | (sensors_clean[col] > upper_bound)
                num_outliers = outliers.sum()
                pct_outliers = (num_outliers / sensors_clean[col].notna().sum()) * 100
                
                outlier_summary.append({
                    'Columna': col,
                    'Min': sensors_clean[col].min(),
                    'Max': sensors_clean[col].max(),
                    'Q1': Q1,
                    'Q3': Q3,
                    'Outliers': num_outliers,
                    'Pct_Outliers': pct_outliers,
                    'Rango_Normal': f"[{lower_bound:.2f}, {upper_bound:.2f}]"
                })
        
        # Mostrar resumen de outliers
        outlier_df = pd.DataFrame(outlier_summary)
        outlier_df_sorted = outlier_df.sort_values('Pct_Outliers', ascending=False)
        
        print(f"\n   📊 Resumen de Outliers (Top 10):")
        print(f"   {'Columna':<25} | {'Min':>10} | {'Max':>10} | {'Outliers':>8} | {'%':>6}")
        print(f"   {'-'*70}")
        
        for _, row in outlier_df_sorted.head(10).iterrows():
            col_name = str(row['Columna'])[:24]
            print(f"   {col_name:<25} | {row['Min']:>10.2f} | {row['Max']:>10.2f} | {row['Outliers']:>8,} | {row['Pct_Outliers']:>5.1f}%")
    
    # 4. Imputación inteligente de valores faltantes
    print(f"\n🔄 Paso 4: Imputación de valores faltantes...")
    
    # Recalcular estadísticas de valores faltantes después de la limpieza
    missing_after_cleaning = sensors_clean.isnull().sum()
    cols_with_missing = missing_after_cleaning[missing_after_cleaning > 0].index.tolist()
    
    if cols_with_missing:
        print(f"   📊 Columnas con valores faltantes: {len(cols_with_missing)}")
        
        for col in cols_with_missing[:10]:  # Procesar las primeras 10
            if col in numeric_cols:
                # Para columnas numéricas, usar mediana
                median_val = sensors_clean[col].median()
                missing_count = sensors_clean[col].isnull().sum()
                sensors_clean[col].fillna(median_val, inplace=True)
                print(f"   ✅ {col[:30]:<30}: {missing_count:,} valores → mediana ({median_val:.2f})")
            elif sensors_clean[col].dtype == 'object':
                # Para columnas de texto, usar moda
                mode_val = sensors_clean[col].mode()
                if len(mode_val) > 0:
                    missing_count = sensors_clean[col].isnull().sum()
                    sensors_clean[col].fillna(mode_val.iloc[0], inplace=True)
                    print(f"   ✅ {col[:30]:<30}: {missing_count:,} valores → moda ({str(mode_val.iloc[0])[:20]})")
        
        if len(cols_with_missing) > 10:
            print(f"   ... y {len(cols_with_missing) - 10} columnas más procesadas")
    else:
        print(f"   ✅ No hay valores faltantes después de la limpieza inicial")
    
    print(f"   📊 Dataset después del paso 4: {sensors_clean.shape[0]:,} filas × {sensors_clean.shape[1]} columnas")
    
    # 5. Resumen final de la limpieza
    print(f"\n📋 RESUMEN DE LA LIMPIEZA:")
    print("=" * 40)
    print(f"📊 Dataset original:    {sensors_df.shape[0]:,} filas × {sensors_df.shape[1]} columnas")
    print(f"📊 Dataset limpio:      {sensors_clean.shape[0]:,} filas × {sensors_clean.shape[1]} columnas")
    print(f"📉 Filas eliminadas:    {sensors_df.shape[0] - sensors_clean.shape[0]:,} ({((sensors_df.shape[0] - sensors_clean.shape[0])/sensors_df.shape[0])*100:.1f}%)")
    print(f"📉 Columnas eliminadas: {sensors_df.shape[1] - sensors_clean.shape[1]} ({((sensors_df.shape[1] - sensors_clean.shape[1])/sensors_df.shape[1])*100:.1f}%)")
    
    # Verificar calidad final
    final_missing = sensors_clean.isnull().sum().sum()
    total_cells = sensors_clean.shape[0] * sensors_clean.shape[1]
    final_missing_pct = (final_missing / total_cells) * 100
    
    print(f"💧 Valores faltantes finales: {final_missing:,} ({final_missing_pct:.2f}% del dataset)")
    print(f"💾 Memoria utilizada: {sensors_clean.memory_usage(deep=True).sum() / (1024*1024):.2f} MB")
    
    # Guardar versión limpia
    print(f"\n💾 Dataset limpio guardado como 'sensors_clean'")
    
else:
    print("❌ No hay datos de sensores disponibles para limpieza")

---

## 3. 📊 Análisis Estadístico Descriptivo

El análisis estadístico descriptivo nos proporciona una comprensión fundamental de las características de nuestros datos. Examinaremos las medidas de tendencia central, dispersión y forma de las distribuciones para cada variable del motocompresor.

### 📈 Estadísticas Generales del Dataset

In [ ]:
# 📊 Análisis estadístico descriptivo completo
if 'sensors_clean' in locals() and sensors_clean is not None:
    print("📊 ANÁLISIS ESTADÍSTICO DESCRIPTIVO")
    print("=" * 50)
    
    # Seleccionar solo columnas numéricas para análisis estadístico
    numeric_cols = sensors_clean.select_dtypes(include=[np.number]).columns.tolist()
    metadata_cols = ['archivo_origen', 'indice_archivo', 'fecha_archivo']
    analysis_cols = [col for col in numeric_cols if col not in metadata_cols]
    
    print(f"📊 Total de variables numéricas: {len(analysis_cols)}")
    print(f"📊 Total de observaciones: {len(sensors_clean):,}")
    
    if analysis_cols:
        # Dataset numérico para análisis
        numeric_data = sensors_clean[analysis_cols]
        
        # Estadísticas descriptivas básicas
        print(f"\n📈 ESTADÍSTICAS DESCRIPTIVAS BÁSICAS:")
        print("-" * 60)
        
        desc_stats = numeric_data.describe()
        
        # Mostrar estadísticas para variables más importantes (primeras 10)
        key_variables = analysis_cols[:10]
        
        print(f"\n{'Variable':<25} | {'Count':>8} | {'Mean':>10} | {'Std':>10} | {'Min':>10} | {'Max':>10}")
        print("-" * 95)
        
        for col in key_variables:
            if col in desc_stats.columns:
                stats = desc_stats[col]
                col_name = col[:24] if len(col) > 24 else col
                print(f"{col_name:<25} | {stats['count']:>8.0f} | {stats['mean']:>10.2f} | {stats['std']:>10.2f} | {stats['min']:>10.2f} | {stats['max']:>10.2f}")
        
        # Estadísticas adicionales calculadas
        print(f"\n📊 ESTADÍSTICAS ADICIONALES:")
        print("-" * 50)
        
        additional_stats = []
        
        for col in key_variables:
            if sensors_clean[col].notna().sum() > 0:
                data_col = sensors_clean[col].dropna()
                
                # Calcular estadísticas adicionales
                coef_var = (data_col.std() / data_col.mean()) * 100 if data_col.mean() != 0 else 0
                skewness = stats.skew(data_col)
                kurtosis = stats.kurtosis(data_col)
                
                additional_stats.append({
                    'Variable': col[:20],
                    'Coef_Variacion': coef_var,
                    'Asimetria': skewness,
                    'Curtosis': kurtosis,
                    'Rango': data_col.max() - data_col.min(),
                    'IQR': data_col.quantile(0.75) - data_col.quantile(0.25)
                })
        
        if additional_stats:
            add_stats_df = pd.DataFrame(additional_stats)
            
            print(f"{'Variable':<20} | {'CV(%)':>8} | {'Asimetría':>10} | {'Curtosis':>10} | {'Rango':>12}")
            print("-" * 75)
            
            for _, row in add_stats_df.iterrows():
                print(f"{row['Variable']:<20} | {row['Coef_Variacion']:>7.1f}% | {row['Asimetria']:>10.3f} | {row['Curtosis']:>10.3f} | {row['Rango']:>12.2f}")
        
        # Interpretación de estadísticas clave
        print(f"\n🔍 INTERPRETACIÓN DE ESTADÍSTICAS CLAVE:")
        print("-" * 50)
        
        interpretations = []
        
        for _, row in add_stats_df.iterrows():
            var_name = row['Variable']
            cv = row['Coef_Variacion']
            skew = row['Asimetria']
            kurt = row['Curtosis']
            
            # Interpretación del coeficiente de variación
            if cv < 10:
                cv_interp = "Baja variabilidad"
            elif cv < 25:
                cv_interp = "Variabilidad moderada"
            else:
                cv_interp = "Alta variabilidad"
            
            # Interpretación de asimetría
            if abs(skew) < 0.5:
                skew_interp = "Simétrica"
            elif abs(skew) < 1:
                skew_interp = "Moderadamente sesgada"
            else:
                skew_interp = "Altamente sesgada"
            
            # Interpretación de curtosis
            if abs(kurt) < 0.5:
                kurt_interp = "Normal"
            elif kurt > 0.5:
                kurt_interp = "Leptocúrtica (colas pesadas)"
            else:
                kurt_interp = "Platicúrtica (colas ligeras)"
            
            interpretations.append({
                'Variable': var_name,
                'Variabilidad': cv_interp,
                'Distribución': skew_interp,
                'Forma': kurt_interp
            })
        
        for interp in interpretations[:8]:  # Mostrar las primeras 8
            print(f"📊 {interp['Variable']:<20}: {interp['Variabilidad']:<20} | {interp['Distribución']:<20} | {interp['Forma']}")
        
        # Identificar variables más relevantes para mantenimiento predictivo
        print(f"\n🎯 VARIABLES MÁS RELEVANTES PARA MANTENIMIENTO PREDICTIVO:")
        print("-" * 65)
        
        # Buscar variables relacionadas con parámetros críticos del motocompresor
        critical_keywords = [
            'rpm', 'presion', 'temperatura', 'temp', 'pressure', 'vibra', 
            'aceite', 'oil', 'cilindro', 'cylinder', 'motor', 'compresor',
            'descarga', 'succion', 'refrigerante', 'coolant'
        ]
        
        critical_vars = []
        for col in analysis_cols:
            col_lower = col.lower()
            for keyword in critical_keywords:
                if keyword in col_lower:
                    critical_vars.append(col)
                    break
        
        # Remover duplicados manteniendo orden
        critical_vars = list(dict.fromkeys(critical_vars))
        
        if critical_vars:
            print(f"✅ Variables críticas identificadas: {len(critical_vars)}")
            for i, var in enumerate(critical_vars[:15], 1):  # Mostrar las primeras 15
                var_stats = desc_stats[var] if var in desc_stats.columns else None
                if var_stats is not None:
                    print(f"   {i:2d}. {var:<35} | Media: {var_stats['mean']:>8.2f} | Std: {var_stats['std']:>8.2f}")
            
            if len(critical_vars) > 15:
                print(f"   ... y {len(critical_vars) - 15} variables críticas más")
        else:
            print(f"⚠️ No se identificaron variables críticas con los criterios establecidos")
            print(f"📊 Mostrando las primeras 10 variables numéricas disponibles:")
            for i, var in enumerate(analysis_cols[:10], 1):
                var_stats = desc_stats[var] if var in desc_stats.columns else None
                if var_stats is not None:
                    print(f"   {i:2d}. {var:<35} | Media: {var_stats['mean']:>8.2f} | Std: {var_stats['std']:>8.2f}")
        
        # Guardar variables críticas para análisis posteriores
        critical_variables = critical_vars if critical_vars else analysis_cols[:15]
        print(f"\n💾 Variables seleccionadas para análisis detallado: {len(critical_variables)}")
        
    else:
        print("❌ No hay variables numéricas disponibles para análisis estadístico")
        critical_variables = []
        
else:
    print("❌ No hay datos limpios disponibles para análisis estadístico")
    critical_variables = []

### 📊 Visualización de Estadísticas Descriptivas

Las visualizaciones nos ayudan a comprender mejor la distribución y características de nuestras variables más importantes.

In [ ]:
# 📊 Visualización de estadísticas descriptivas
if 'sensors_clean' in locals() and sensors_clean is not None and 'critical_variables' in locals():
    print("📊 VISUALIZACIÓN DE ESTADÍSTICAS DESCRIPTIVAS")
    print("=" * 55)
    
    if critical_variables:
        # Seleccionar las primeras 8 variables críticas para visualización
        viz_vars = critical_variables[:8]
        
        # 1. Resumen estadístico visual
        print(f"\n📈 Generando visualizaciones para {len(viz_vars)} variables críticas...")
        
        # Crear figura con múltiples subplots
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('📊 Resumen Estadístico de Variables Críticas del Motocompresor C-5080', 
                     fontsize=16, fontweight='bold', y=0.95)
        
        # Subplot 1: Comparación de medias
        ax1 = axes[0, 0]
        means = [sensors_clean[var].mean() for var in viz_vars if sensors_clean[var].notna().sum() > 0]
        var_names = [var for var in viz_vars if sensors_clean[var].notna().sum() > 0]
        
        if means:
            bars1 = ax1.bar(range(len(means)), means, color='steelblue', alpha=0.7)
            ax1.set_title('📊 Valores Promedio por Variable', fontweight='bold')
            ax1.set_xlabel('Variables')
            ax1.set_ylabel('Valor Promedio')
            ax1.set_xticks(range(len(var_names)))
            ax1.set_xticklabels([name[:15] + '...' if len(name) > 15 else name for name in var_names], 
                               rotation=45, ha='right')
            ax1.grid(axis='y', alpha=0.3)
            
            # Agregar valores en las barras
            for bar, mean_val in zip(bars1, means):
                height = bar.get_height()
                ax1.text(bar.get_x() + bar.get_width()/2., height,
                        f'{mean_val:.1f}', ha='center', va='bottom', fontsize=9)
        
        # Subplot 2: Comparación de coeficientes de variación
        ax2 = axes[0, 1]
        cvs = []
        cv_vars = []
        
        for var in viz_vars:
            if sensors_clean[var].notna().sum() > 0:
                mean_val = sensors_clean[var].mean()
                std_val = sensors_clean[var].std()
                if mean_val != 0:
                    cv = (std_val / mean_val) * 100
                    cvs.append(cv)
                    cv_vars.append(var)
        
        if cvs:
            colors = ['red' if cv > 25 else 'orange' if cv > 10 else 'green' for cv in cvs]
            bars2 = ax2.bar(range(len(cvs)), cvs, color=colors, alpha=0.7)
            ax2.set_title('📊 Coeficiente de Variación (%)', fontweight='bold')
            ax2.set_xlabel('Variables')
            ax2.set_ylabel('CV (%)')
            ax2.set_xticks(range(len(cv_vars)))
            ax2.set_xticklabels([name[:15] + '...' if len(name) > 15 else name for name in cv_vars], 
                               rotation=45, ha='right')
            ax2.grid(axis='y', alpha=0.3)
            
            # Líneas de referencia
            ax2.axhline(y=10, color='orange', linestyle='--', alpha=0.5, label='CV = 10%')
            ax2.axhline(y=25, color='red', linestyle='--', alpha=0.5, label='CV = 25%')
            ax2.legend()
        
        # Subplot 3: Distribución de rangos
        ax3 = axes[1, 0]
        ranges = []
        range_vars = []
        
        for var in viz_vars:
            if sensors_clean[var].notna().sum() > 0:
                var_range = sensors_clean[var].max() - sensors_clean[var].min()
                ranges.append(var_range)
                range_vars.append(var)
        
        if ranges:
            bars3 = ax3.bar(range(len(ranges)), ranges, color='forestgreen', alpha=0.7)
            ax3.set_title('📊 Rango de Valores por Variable', fontweight='bold')
            ax3.set_xlabel('Variables')
            ax3.set_ylabel('Rango (Max - Min)')
            ax3.set_xticks(range(len(range_vars)))
            ax3.set_xticklabels([name[:15] + '...' if len(name) > 15 else name for name in range_vars], 
                               rotation=45, ha='right')
            ax3.grid(axis='y', alpha=0.3)
        
        # Subplot 4: Comparación de asimetría
        ax4 = axes[1, 1]
        skewnesses = []
        skew_vars = []
        
        for var in viz_vars:
            if sensors_clean[var].notna().sum() > 10:  # Necesitamos suficientes datos
                skew_val = stats.skew(sensors_clean[var].dropna())
                skewnesses.append(skew_val)
                skew_vars.append(var)
        
        if skewnesses:
            colors = ['red' if abs(s) > 1 else 'orange' if abs(s) > 0.5 else 'green' for s in skewnesses]
            bars4 = ax4.bar(range(len(skewnesses)), skewnesses, color=colors, alpha=0.7)
            ax4.set_title('📊 Asimetría de las Distribuciones', fontweight='bold')
            ax4.set_xlabel('Variables')
            ax4.set_ylabel('Coeficiente de Asimetría')
            ax4.set_xticks(range(len(skew_vars)))
            ax4.set_xticklabels([name[:15] + '...' if len(name) > 15 else name for name in skew_vars], 
                               rotation=45, ha='right')
            ax4.grid(axis='y', alpha=0.3)
            
            # Línea de referencia en 0 (distribución simétrica)
            ax4.axhline(y=0, color='black', linestyle='-', alpha=0.5, label='Simétrica')
            ax4.axhline(y=0.5, color='orange', linestyle='--', alpha=0.5, label='Moderada')
            ax4.axhline(y=-0.5, color='orange', linestyle='--', alpha=0.5)
            ax4.legend()
        
        plt.tight_layout()
        plt.show()
        
        # 2. Tabla resumen de estadísticas críticas
        print(f"\n📋 TABLA RESUMEN DE ESTADÍSTICAS CRÍTICAS:")
        print("=" * 90)
        
        summary_data = []
        for var in viz_vars:
            if sensors_clean[var].notna().sum() > 0:
                data_col = sensors_clean[var].dropna()
                
                summary_data.append({
                    'Variable': var[:25],
                    'N': len(data_col),
                    'Media': data_col.mean(),
                    'Mediana': data_col.median(),
                    'Desv_Std': data_col.std(),
                    'Min': data_col.min(),
                    'Max': data_col.max(),
                    'Q1': data_col.quantile(0.25),
                    'Q3': data_col.quantile(0.75)
                })
        
        if summary_data:
            summary_df = pd.DataFrame(summary_data)
            
            print(f"{'Variable':<25} | {'N':>6} | {'Media':>10} | {'Mediana':>10} | {'Desv_Std':>10} | {'Min':>10} | {'Max':>10}")
            print("-" * 105)
            
            for _, row in summary_df.iterrows():
                print(f"{row['Variable']:<25} | {row['N']:>6,} | {row['Media']:>10.2f} | {row['Mediana']:>10.2f} | {row['Desv_Std']:>10.2f} | {row['Min']:>10.2f} | {row['Max']:>10.2f}")
        
        print(f"\n🔍 INTERPRETACIONES CLAVE:")
        print("-" * 30)
        print(f"• Variables con alta variabilidad (CV > 25%): Requieren atención especial")
        print(f"• Variables asimétricas (|skew| > 1): Pueden necesitar transformación")
        print(f"• Rangos amplios: Indicativos de condiciones operacionales diversas")
        print(f"• Medias vs. Medianas: Diferencias grandes sugieren presencia de outliers")
        
    else:
        print("⚠️ No hay variables críticas disponibles para visualización")
        
else:
    print("❌ No hay datos disponibles para visualización de estadísticas descriptivas")

---

**📄 Nota:** Este notebook está siendo desarrollado como parte de un análisis completo de mantenimiento predictivo. Las siguientes secciones incluirán análisis univariado, bivariado, series temporales y correlación con eventos de falla.

---

## 🔄 Estado del Análisis

**✅ Completado:**
- Carga e inspección inicial de datos
- Limpieza y validación de datos
- Análisis estadístico descriptivo

**⏳ En desarrollo:**
- Análisis univariado (distribuciones)
- Análisis bivariado (correlaciones)
- Análisis de series temporales
- Integración con historial de fallas
- Conclusiones y recomendaciones